In [11]:
import json
import os

import openai
from langchain.tools import OpenAPISpec
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
MP_API_KEY = os.getenv("MP_API_KEY", None)
openai.api_key = OPENAI_API_KEY

# spec = OpenAPISpec.from_url("https://api.materialsproject.org/openapi.json")
spec = OpenAPISpec.from_file("../materiathena/apis/mp_openapi.json")

In [68]:
# from langchain.llms.openai import OpenAI
# from langchain.agents.agent_toolkits.openapi import planner
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

In [24]:
from openapi_schema_pydantic import PathItem, Operation, Parameter

# TODO: construct a class

def get_endpoints(spec: OpenAPISpec):
    endpoints = [
        (route, method, operation)
        for route, paths in spec.paths.items()
        for method, operation in paths
        if method in ["get", "post"]
    ]
    return endpoints

def get_functions(spec: OpenAPISpec):
    endpoints = get_endpoints(spec)
    functions = []
    for route, method, operation in endpoints:
        if operation is None:
            continue

        name = operation.operationId if len(operation.operationId) <= 64 else operation.operationId[:64]

        description = operation.description or operation.summary

        properties = {
            property.name: {
                "type": property.param_schema.type,
                "description": property.description if property.description is not None else "",
                **({"enum": property.param_schema.enum} if property.param_schema.enum is not None else {}),
            }
            for property in operation.parameters if property.param_schema.type is not None
        }

        required = [
            property.name
            for property in operation.parameters
            if property.required
        ]
        functions.append({
            "name": name,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": properties,
            },
            **({"required": required} if required else {})
        })

    return functions

functions = get_functions(spec)

print(len(functions))

98


In [2]:
# import json
# import os

# import openai
# from dotenv import load_dotenv
# from mp_api.client import MPRester

# from langchain.tools import APIOperation, OpenAPISpec

# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
# MP_API_KEY = os.getenv("MP_API_KEY", None)
# openai.api_key = OPENAI_API_KEY

# spec = OpenAPISpec.from_url("https://api.materialsproject.org/openapi.json")
# operation = APIOperation.from_openapi_spec(
#     spec, "/materials/summary/{material_id}/", "get"
# )

# # from langchain.chat_models import ChatOpenAI, ChatAnthropic
# # from langchain.schema import HumanMessage, AIMessage, ChatMessage
# # from langchain.tools import format_tool_to_openai_function

# # llm = ChatAnthropic(anthropic_api_key=)
# # llm = ChatOpenAI(model='gpt-4-0613', temperature=0.5, openai_api_key=OPENAI_API_KEY, client=)


# class LLMaterialsAgent:
#     def __init__(self, mp_api_key=MP_API_KEY, openai_api_key=OPENAI_API_KEY):
#         # Initialize the Materials Project API
#         self.mpr = MPRester(mp_api_key)
#         # Initialize the OpenAI API
#         openai.api_key = openai_api_key

#     def get_materials_data(self, query_params):
#         # Retrieve data from the Materials Project using the MPRester class
#         data = self.mpr.summary.search(**query_params)
#         return data

#     def run_conversation(self, user_input):
#         # Step 1: send the conversation and available functions to GPT
#         messages = [{"role": "user", "content": user_input}]
#         functions = [
#             {
#                 "name": "get_materials_data",
#                 "description": "Get materials data from the Materials Project",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "elements": {
#                             "type": "array",
#                             "items": {"type": "string"},
#                             "description": "Elements to query, e.g., ['Si', 'O']",
#                         },
#                         "band_gap": {
#                             "type": "array",
#                             "items": {"type": "number"},
#                             "description": "Range of band gap values, e.g., [0.5, 1.0]",
#                         },
#                         "fields": {
#                             "type": "array",
#                             "items": {"type": "string"},
#                             "description": f"Fields to return, including {self.mpr.summary.available_fields}",
#                         }
#                         # "limit": {
#                         #     "type": "integer",
#                         #     "description": "Number of materials to retrieve, e.g., 5",
#                         # },
#                     },
#                     "required": ["elements", "band_gap"],
#                 },
#             }
#         ]
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo-0613",
#             messages=messages,
#             functions=functions,
#             function_call="auto",  # auto is default, but we'll be explicit
#         )
#         response_message = response["choices"][0]["message"]

#         # Step 2: check if GPT wanted to call a function
#         if response_message.get("function_call"):
#             # Step 3: call the function
#             # Note: the JSON response may not always be valid; be sure to handle errors
#             available_functions = {
#                 "get_materials_data": self.get_materials_data,
#             }  # only one function in this example, but you can have multiple
#             function_name = response_message["function_call"]["name"]
#             function_to_call = available_functions[function_name]
#             function_args = json.loads(response_message["function_call"]["arguments"])
#             function_response = function_to_call(query_params=function_args)

#             # breakpoint()

#             # Step 4: send the info on the function call and function response to GPT
#             messages.append(
#                 response_message
#             )  # extend conversation with assistant's reply
#             # breakpoint()
#             messages.append(
#                 {
#                     "role": "function",
#                     "name": function_name,
#                     "content": json.dumps(function_response[0]),
#                     # "content": function_response,
#                 }
#             )  # extend conversation with function response
#             # breakpoint()
#             second_response = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo-0613",
#                 messages=messages,
#             )  # get a new response from GPT where it can see the function response
#             # breakpoint()
#             return second_response

In [3]:
# ll_agent = LLMaterialsAgent()


/global/homes/c/cyrusyc/.conda/envs/dev/lib/python3.11/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [39]:
# functions = [
#     {
#         "name": "get_by_key_materials_bonds__material_id___get",
#         "description": "Get a BondingDoc document by by material_id",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "material_id": {"type": "string", "description": ""},
#                 "_fields": {
#                     "type": "string",
#                     "description": "Fields to project from BondingDoc as a list of comma separated strings.",
#                 },
#                 "_all_fields": {
#                     "type": "boolean",
#                     "description": "Include all fields.",
#                 },
#             },
#             "required": ["material_id"],
#         },
#     },
#     {
#         "name": "get_by_key_materials_absorption__material_id___get",
#         "description": "Get a AbsorptionDoc document by by material_id",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "material_id": {"type": "string", "description": ""},
#                 "_fields": {
#                     "type": "string",
#                     "description": "Fields to project from AbsorptionDoc as a list of comma separated strings.",
#                 },
#                 "_all_fields": {
#                     "type": "boolean",
#                     "description": "Include all fields.",
#                 },
#             },
#             "required": ["material_id"],
#         },
#     },
#     {
#         "name": "get_by_key_materials_provenance__material_id___get",
#         "description": "Get a ProvenanceDoc document by by material_id",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "material_id": {"type": "string", "description": ""},
#                 "_fields": {
#                     "type": "string",
#                     "description": "Fields to project from ProvenanceDoc as a list of comma separated strings.",
#                 },
#                 "_all_fields": {
#                     "type": "boolean",
#                     "description": "Include all fields.",
#                 },
#             },
#             "required": ["material_id"],
#         },
#     },
#     {
#         "name": "get_by_key_materials_electronic_structure__material_id___get",
#         "description": "Get a ElectronicStructureDoc document by by material_id",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "material_id": {"type": "string", "description": ""},
#                 "_fields": {
#                     "type": "string",
#                     "description": "Fields to project from ElectronicStructureDoc as a list of comma separated strings.",
#                 },
#                 "_all_fields": {
#                     "type": "boolean",
#                     "description": "Include all fields.",
#                 },
#             },
#             "required": ["material_id"],
#         },
#     },
#     {
#         "name": "get_by_key_materials_piezoelectric__material_id___get",
#         "description": "Get a PiezoelectricDoc document by by material_id",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "material_id": {"type": "string", "description": ""},
#                 "_fields": {
#                     "type": "string",
#                     "description": "Fields to project from PiezoelectricDoc as a list of comma separated strings.",
#                 },
#                 "_all_fields": {
#                     "type": "boolean",
#                     "description": "Include all fields.",
#                 },
#             },
#             "required": ["material_id"],
#         },
#     },
# ]

In [38]:
# user_input = input("Please enter your natural language query: ")

user_input = "Is UCl3 a stable material according to the thermodynamic hull on Materials Project?"

messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Only use the functions you have been provided with."},
    {"role": "system", "content": "Avoid using search_mpcomplete__get and retrieve only relevant data useful to complete response."},
    {"role": "user", "content": user_input}
]


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 36296 tokens (80 in the messages, 36216 in the functions). Please reduce the length of the messages or functions.

In [32]:
from tqdm.auto import tqdm
passed_functions = []
for n in tqdm(range(len(functions))):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            functions=[functions[n]],
            function_call="auto",  # auto is default, but we'll be explicit
        )
        response_message = response["choices"][0]["message"]
    except Exception as e:
        print(functions[n]["name"])
        continue
    passed_functions.append(functions[n])


  0%|          | 0/98 [00:00<?, ?it/s]

search_materials_synthesis__get


In [37]:
list(map(lambda x: x['name'], functions)).index('search_materials_synthesis__get')

functions.pop(47)

{'name': 'search_materials_synthesis__get',
 'description': 'Get SynthesisSearchResultModel documents',
 'parameters': {'type': 'object',
  'properties': {'keywords': {'type': 'string',
    'description': 'Comma delimited string keywords to search synthesis paragraph text with.'},
   'synthesis_type': {'type': 'array',
    'description': 'Type of synthesis to include.'},
   'target_formula': {'type': 'string',
    'description': 'Chemical formula of the target material.'},
   'precursor_formula': {'type': 'string',
    'description': 'Chemical formula of the precursor material.'},
   'operations': {'type': 'array',
    'description': 'List of operations that syntheses must have.'},
   'condition_heating_temperature_min': {'type': 'number',
    'description': 'Minimal heating temperature.'},
   'condition_heating_temperature_max': {'type': 'number',
    'description': 'Maximal heating temperature.'},
   'condition_heating_time_min': {'type': 'number',
    'description': 'Minimal heating

In [40]:
import json

with open("passed_functions.json", "w") as f:
    json.dump(functions, f, indent=4)

functions

[{'name': 'search_mpcomplete__get',
  'description': 'Get MPCompleteDoc data',
  'parameters': {'type': 'object',
   'properties': {'public_name': {'type': 'string', 'description': ''},
    'public_email': {'type': 'string', 'description': ''},
    '_page': {'type': 'integer',
     'description': 'Page number to request (takes precedent over _limit and _skip).'},
    '_per_page': {'type': 'integer',
     'description': 'Number of entries to show per page (takes precedent over _limit and _skip). Limited to 1000.'},
    '_skip': {'type': 'integer',
     'description': 'Number of entries to skip in the search.'},
    '_limit': {'type': 'integer',
     'description': 'Max number of entries to return in a single query. Limited to 1000.'},
    'last_updated': {'type': 'string',
     'description': 'Minimum datetime of status update for submission'}}}},
 {'name': 'post_data_mpcomplete__post',
  'description': 'Post MPCompleteDoc data',
  'parameters': {'type': 'object',
   'properties': {'pub

In [20]:
available_functions = {
    "get_materials_data": ll_agent.get_materials_data,
}  # only one function in this example, but you can have multiple
function_name = response_message["function_call"]["name"]
function_to_call = available_functions[function_name]
function_args = json.loads(response_message["function_call"]["arguments"])
function_response = function_to_call(query_params=function_args)

Retrieving SummaryDoc documents:   0%|          | 0/724 [00:00<?, ?it/s]

In [13]:
from monty.json import MontyEncoder, MontyDecoder

# json.dumps(function_response[0], cls=MontyEncoder)
# json.dumps(function_response[0].dict(), indent=4, cls=MontyDecoder)
# function_response[0]
# MontyEncoder().encode(function_response[0])
# MontyDecoder().decode(function_response[0].__str__())

In [21]:
function_response[0]

MPDataDoc<SummaryDoc>(
formula_pretty='PH9C(NO)4',
band_gap=5.0001999999999995,
fields_not_requested=['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_r

In [22]:
messages.append(
    {
        "role": "function",
        "name": function_name,
        # "content": json.dumps(function_response[0]),
        "content": function_response[0].__str__(),
    }
)

In [23]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)  # get a new response from GPT where it can see the function response

In [24]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7u22fj6BxWpNJiiFjUz6XcfzcxCFl at 0x7f5e013b86b0> JSON: {
  "id": "chatcmpl-7u22fj6BxWpNJiiFjUz6XcfzcxCFl",
  "object": "chat.completion",
  "created": 1693588573,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "One material that meets your criteria is PH9C(NO)4, which has a bandgap of 5.0002 eV."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 464,
    "completion_tokens": 28,
    "total_tokens": 492
  }
}

In [59]:
# from emmet.core.summary import SummaryDoc
# from monty.serialization import 


TypeError: __init__() takes exactly 1 positional argument (2 given)

In [8]:
from pymatgen.core import Structure, Element
import datetime


def serialize_nested_object(obj):
    if isinstance(obj, dict):
        serialized_dict = {}
        for key, value in obj.items():
            serialized_dict[key] = serialize_nested_object(value)
        return serialized_dict
    elif isinstance(obj, list):
        serialized_list = []
        for item in obj:
            serialized_list.append(serialize_nested_object(item))
        return serialized_list
    elif isinstance(obj, (datetime.datetime, Element)):
        return str(obj)
    elif isinstance(obj, (str, int, float)):
        return obj
    else:
        # Handle other data types or custom objects as needed
        return str(obj)


serialize_nested_object(function_response[0].dict())

{'builder_meta': {'emmet_version': '0.38.6',
  'pymatgen_version': '2022.10.22',
  'pull_request': 825,
  'database_version': '2022.10.28',
  'build_date': '2022-10-28 00:56:47.317000'},
 'nsites': 76,
 'elements': ['C', 'H', 'N', 'O', 'P'],
 'nelements': 5,
 'composition': 'P4 H36 C4 N16 O16',
 'composition_reduced': 'P1 H9 C1 N4 O4',
 'formula_pretty': 'PH9C(NO)4',
 'formula_anonymous': 'ABC4D4E9',
 'chemsys': 'C-H-N-O-P',
 'volume': 729.1804951382222,
 'density': 1.5674917207978354,
 'density_atomic': 9.594480199187135,
 'symmetry': {'crystal_system': 'Triclinic',
  'symbol': 'P-1',
  'number': 2,
  'point_group': '-1',
  'symprec': 0.1,
  'version': '1.16.2'},
 'property_name': 'summary',
 'material_id': MPID(mp-604964),
 'deprecated': False,
 'deprecation_reasons': 'None',
 'last_updated': '2022-10-28 00:56:47.317000',
 'origins': [],
 'warnings': [],
 'structure': 'Full Formula (P4 H36 C4 N16 O16)\nReduced Formula: PH9C(NO)4\nabc   :   6.904773   7.560272  14.335972\nangles:  87.

In [60]:
from monty.json import MontyEncoder

encoder = MontyEncoder()
a = encoder.encode(function_response[0])

TypeError: 'str' object does not support item assignment

In [63]:
from monty.json import MontyDecoder, MontyEncoder

decoder = MontyDecoder()
decoder.decode(function_response[0].dict())
# encoder = MontyEncoder()
# encoder.encode()
# json.dumps(function_response[0])

TypeError: the JSON object must be str, bytes or bytearray, not dict